In [27]:
%run Functions.py
import warnings
warnings.filterwarnings('ignore')

## M1

In [38]:
train = pd.read_csv('train_c.csv')
test = pd.read_csv('test_c.csv')

In [39]:
train = train.sample(frac=1).reset_index(drop=True)

In [40]:
label1 = train['price_doc']*0.95 + 10.5

In [41]:
col1 = list(test.columns)[2:]

In [42]:
rounds1 = 372
params1 = {
    'eta': 0.05, ## Try 0.01,3,5
    'max_depth': 5,## Try 4,5,6
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
} 

In [43]:
model1_input,model1_predict = get_xgb_stack_data(params1,rounds1,train,col1,label1,test)

TRAIN: [ 6095  6096  6097 ..., 30468 30469 30470] TEST: [   0    1    2 ..., 6092 6093 6094]
TRAIN: [    0     1     2 ..., 30468 30469 30470] TEST: [ 6095  6096  6097 ..., 12186 12187 12188]
TRAIN: [    0     1     2 ..., 30468 30469 30470] TEST: [12189 12190 12191 ..., 18280 18281 18282]
TRAIN: [    0     1     2 ..., 30468 30469 30470] TEST: [18283 18284 18285 ..., 24374 24375 24376]
TRAIN: [    0     1     2 ..., 24374 24375 24376] TEST: [24377 24378 24379 ..., 30468 30469 30470]
Start Training


## M2

In [59]:
train = pd.read_csv('train_clean.csv')
test = pd.read_csv('test_clean.csv')
macro = pd.read_csv('macro_c.csv')

In [60]:
train,test = get_additional_features(train,test,macro)
col = list(test.columns)[2:]
train = train.sample(frac=1).reset_index(drop=True)

In [61]:
train['life_sq_ratio'] = train['life_sq']/train['full_sq']
train['kitch_sq_ratio'] = train['kitch_sq']/train['full_sq']
test['life_sq_ratio'] = test['life_sq']/test['full_sq']
test['kitch_sq_ratio'] = test['kitch_sq']/test['full_sq']

In [62]:
Price_sqm = [x/y for x,y in zip(list(train['price_doc']*0.95+10),list(train['full_sq']))]
col = list(test.columns)[2:]
col = [each for each in col if each not in ['full_sq','life_sq','kitch_sq']]
train['price_sqm'] = Price_sqm
label = train['price_sqm']

In [32]:
train2 = train
feature2 = col
label2 = label
params2 = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}
rounds2 = 400

In [34]:
model2_input,model2_predict = get_xgb_stack_data(params2,rounds2,train2,feature2,label2,test)

TRAIN: [ 5853  5854  5855 ..., 29259 29260 29261] TEST: [   0    1    2 ..., 5850 5851 5852]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [ 5853  5854  5855 ..., 11703 11704 11705]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [11706 11707 11708 ..., 17555 17556 17557]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [17558 17559 17560 ..., 23407 23408 23409]
TRAIN: [    0     1     2 ..., 23407 23408 23409] TEST: [23410 23411 23412 ..., 29259 29260 29261]
Start Training


In [35]:
model2_input_backup = model2_input

In [36]:
model2_input['predicted'] = model2_input['predicted']*model2_input['full_sq']
model2_predict = model2_predict*test['full_sq']
model2_input['label'] = train2['price_doc']*0.95+10

# M3

In [63]:
def quarter_cal(Month):
    if Month>=1 and Month <=3:
        return 'Q1'
    elif Month >3 and Month <=6:
        return 'Q2'
    elif Month >6 and Month <=9:
        return 'Q3'
    else:
        return 'Q4'

In [64]:
train_quarter = [str(x)+quarter_cal(y) for x,y in zip(list(train.timestamp.dt.year),list(train.month))]
test_quarter = [str(x)+quarter_cal(y) for x,y in zip(list(test.timestamp.dt.year),list(test.month))]

In [65]:
train['Quarter'] = train_quarter
test['Quarter'] = test_quarter

In [66]:
quarter_data = pd.read_csv('Quaterly Data.csv',sep=';')[['Quarter','Primary Average']]
train = train.merge(quarter_data,on='Quarter',how='left')
test = test.merge(quarter_data,on='Quarter',how='left')
del train['Quarter']
del test['Quarter']

In [67]:
Price_gap = Price_sqm - train['Primary Average']
train['Gap'] = Price_gap

In [68]:
del train['price_sqm']

In [69]:
params3 = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 5,## Try 4,5,6
   'subsample': 0.7,
   'colsample_bytree': 0.7,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1,
    'lambda':5,
    'min_child_weight':5
}
train3 = train
feature3 = list(test.columns)[2:]
label3 = train['Gap']
round3 = 400
feature3.remove("Primary Average")

In [70]:
model3_input,model3_predict = get_xgb_stack_data(params3,round3,train3,feature3,label3,test)

TRAIN: [ 5853  5854  5855 ..., 29259 29260 29261] TEST: [   0    1    2 ..., 5850 5851 5852]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [ 5853  5854  5855 ..., 11703 11704 11705]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [11706 11707 11708 ..., 17555 17556 17557]
TRAIN: [    0     1     2 ..., 29259 29260 29261] TEST: [17558 17559 17560 ..., 23407 23408 23409]
TRAIN: [    0     1     2 ..., 23407 23408 23409] TEST: [23410 23411 23412 ..., 29259 29260 29261]
Start Training


In [71]:
model3_input_bakup = model3_input

In [72]:
model3_input['predicted'] = (model3_input['predicted'] + model3_input['Primary Average'])*model3_input['full_sq']
model3_input['label'] = train3['price_doc']*0.95+10

In [73]:
model3_predict = (model3_predict+test['Primary Average'])*test['full_sq']

## Together

In [133]:
model1_in = model1_input[['id','predicted','label','timestamp']]
model1_in.columns = ['id','model1','label1','timestamp']

In [134]:
model2_in = model2_input[['id','predicted','label']]
model3_in = model3_input[['id','predicted','label']]
model2_in.columns = ['id','model2','label2']
model3_in.columns = ['id','model3','label3']

In [135]:
model_input = model2_in.merge(model3_in,on='id',how='left')

In [136]:
model_input = model_input.merge(model1_in,on='id',how='left')

In [137]:
model_input = model_input[['id','model1','model2','model3','label1','timestamp']]

In [138]:
model_output = pd.DataFrame({'model1':model1_predict,'model2':model2_predict,'model3':model3_predict})

In [139]:
model_output['timestamp'] = test['timestamp']

In [140]:
cols = list(model_output.columns)
cols.remove('timestamp')

In [141]:
from sklearn.metrics import mean_squared_error as mse
for each in cols:
    print(mse(model_input['label1'],model_input[each])**0.5)

2273785.12717
2285810.27173
2262386.20152


In [142]:
model_input = model_input.sort_values(['id'],ascending=1)

In [143]:
model_input = model_input.reset_index(drop=True)

In [144]:
price = pd.read_csv('moscow_avg_price.csv')

In [145]:
price = price[['timestamp','moscow_avg_price_low_rub']]
price['timestamp'] = pd.to_datetime(price['timestamp'])

In [146]:
model_input = model_input.merge(price,on='timestamp',how='left')
model_output = model_output.merge(price,on = 'timestamp',how = 'left')

In [148]:
features = list(model_output)
features.remove('timestamp')

In [149]:
params = {
   'eta': 0.05, ## Try 0.01,3,5
   'max_depth': 2,## Try 4,5,6
   'subsample': 1,
   'colsample_bytree':1,
   'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

In [153]:
dtrain = xgb.DMatrix(model_input[features],model_input['label1'])
dtest = xgb.DMatrix(model_output[features])
xgb_cv = xgb.cv(params,dtrain,num_boost_round=1000,early_stopping_rounds=20,
     verbose_eval=50, show_stdv=True,seed=42)
print(len(xgb_cv))

[0]	train-rmse:7.94037e+06+13825.6	test-rmse:7.94047e+06+24608.5
[50]	train-rmse:2.27101e+06+35021.8	test-rmse:2.36433e+06+81089.3
[100]	train-rmse:2.11186e+06+34799.9	test-rmse:2.25612e+06+74219.7
103


In [160]:
model_level2 = xgb.train(params,dtrain,num_boost_round=103)

In [161]:
pred = model_level2.predict(dtest)
sub = pd.DataFrame({'id':test['id'],'price_doc':pred})

In [163]:
sub.to_csv('Stacking_Three_Shuffled_Models.csv',index=False)

In [155]:
kf = KFold(n_splits=5)

In [159]:
for train_index,test_index in kf.split(model_input):
    print(test_index)
    Label = model_input.iloc[test_index,:]['label1']
    for each in ['model1','model2','model3']:
        Pred = model_input.iloc[test_index,:][each]
        print(mse(Pred,Label)**0.5)

[   0    1    2 ..., 5850 5851 5852]
2680951.9575
2663748.6802
2675655.17657
[ 5853  5854  5855 ..., 11703 11704 11705]
2175288.18557
2203528.52866
2157402.14502
[11706 11707 11708 ..., 17555 17556 17557]
2167360.14743
2209611.9036
2196438.09484
[17558 17559 17560 ..., 23407 23408 23409]
2056502.33624
2131969.76274
2067661.24259
[23410 23411 23412 ..., 29259 29260 29261]
2237004.74132
2178442.4748
2163009.2999
